In [ ]:
#Para iniciar o programa, é necessário importar alguma bibliotecas que são bastante utilizadas quando se trata 
#de machine learning e manipulação dos dados.

import numpy as np #Biblioteca "matemática"
import pandas as pd #Biblioteca para manipulação e análise de dados
import matplotlib.pyplot as plt #Extenção da biblioteca que faz a pltagem de gráficos e pontos
import seaborn as sns
import os #Funcionalidade simplificadas de sistema operacionais

In [ ]:
# Neste primeiro passo é feito o acesso a pasta do google e é feito a "Montagem" desta pasta 
#https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#Neste segundo momento, é feito o acesso a pasta aonde está o arquivo .CSV que iremos utilizar para fazer as predições em nosso modelo

df=pd.read_csv('/content/drive/MyDrive/meu_projeto/meu_projeto_env/bin/measures_v2.csv', usecols=[0,1,2,3,4,5,6,7,8,9,10,11])


In [ ]:
#Como podemos observar no dataset, existe um grande número de colunas para nossa variável, porém gostariamo de predizer "pm" que é
# a temperatura do rotor. Para isto, colocaremos ela como nosso Target.
df.head()

,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,pm,stator_yoke,ambient,torque
0,-0.450682,18.805172,19.086670,-0.350055,18.293219,0.002866,0.004419,0.000328,24.554214,18.316547,19.850691,0.187101
1,-0.325737,18.818571,19.092390,-0.305803,18.294807,0.000257,0.000606,-0.000785,24.538078,18.314955,19.850672,0.245417
2,-0.440864,18.828770,19.089380,-0.372503,18.294094,0.002355,0.001290,0.000386,24.544693,18.326307,19.850657,0.176615
3,-0.327026,18.835567,19.083031,-0.316199,18.292542,0.006105,0.000026,0.002046,24.554018,18.330833,19.850647,0.238303
4,-0.471150,18.857033,19.082525,-0.332272,18.291428,0.003133,-0.064317,0.037184,24.565397,18.326662,19.850639,0.208197


In [ ]:
#Então pegamos a coluna com os valores de pm e as declaramos como o Target
target = df.pop('pm') #Temperatura do rotor
#Após isto a colocamos é feita a concatenção dela novamente no tabela de dados, porém agora ela se encontra na última coluna de nosso dataframe.
df = pd.concat([df, target], axis=1)
#Aqui é utilizado a função sample para embalhamento dos valores.
df = df.sample(frac=1,random_state=0) #embaralha os dados do dataframe #Ajuda a previnir o overfitting
df.head()

,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,stator_yoke,ambient,torque,pm
372073,41.938923,18.744030,66.684830,-123.478027,46.080647,4749.964355,-187.964111,77.635490,33.169899,20.551407,7.486735e+01,40.030872
766578,-0.431508,59.902590,85.079312,-0.878644,76.299257,0.057160,-2.000745,1.097449,67.216376,24.467134,-2.614878e-06,72.503215
1319224,-1.541598,33.149664,48.669293,-0.333442,45.330586,0.001482,-2.000673,1.097664,39.476950,23.169348,5.436636e-14,43.026863
643478,42.387482,44.949261,104.791174,-123.337533,90.274398,5112.368164,-181.587703,68.767176,72.123242,27.374628,6.631759e+01,74.825417
552128,15.335679,18.755226,113.366333,-130.067474,84.144737,3999.963135,-205.157623,97.281723,57.477020,23.940556,9.412620e+01,89.903763


In [ ]:
#Como pode ser visto, o Index de nosso dataframe estava desorganizado. Utilizando o Reset index foi possível traze-lo para a ordem novamente 
#Tendo o index organizado novamente.
df.reset_index(drop=True, inplace=True) #Faz com que o Index volte a ser o que era antes
df.head()

,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,stator_yoke,ambient,torque,pm
0,41.938923,18.744030,66.684830,-123.478027,46.080647,4749.964355,-187.964111,77.635490,33.169899,20.551407,7.486735e+01,40.030872
1,-0.431508,59.902590,85.079312,-0.878644,76.299257,0.057160,-2.000745,1.097449,67.216376,24.467134,-2.614878e-06,72.503215
2,-1.541598,33.149664,48.669293,-0.333442,45.330586,0.001482,-2.000673,1.097664,39.476950,23.169348,5.436636e-14,43.026863
3,42.387482,44.949261,104.791174,-123.337533,90.274398,5112.368164,-181.587703,68.767176,72.123242,27.374628,6.631759e+01,74.825417
4,15.335679,18.755226,113.366333,-130.067474,84.144737,3999.963135,-205.157623,97.281723,57.477020,23.940556,9.412620e+01,89.903763


In [ ]:
#Neste momento fazemos a divisão de nosso banco de dados em 75% e 25% a partir do quantidade de linhas de df.
split_index=int(len(df) * 0.75)

#Então aqui fazemos uma jogada bem típica de machine learning onde é separado o conjunto de treino e conjunto de teste.
train_df = df[:split_index] #Primeiros 75%
test_df = df[split_index:] #outros 25% restantes

#É possível observar pela saída dos dados que temos exatamente 25% de linhas para o teste e 
#75% de linhas para o treino de nosso algorítmo.
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998112 entries, 0 to 998111
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   u_q             998112 non-null  float64
 1   coolant         998112 non-null  float64
 2   stator_winding  998112 non-null  float64
 3   u_d             998112 non-null  float64
 4   stator_tooth    998112 non-null  float64
 5   motor_speed     998112 non-null  float64
 6   i_d             998112 non-null  float64
 7   i_q             998112 non-null  float64
 8   stator_yoke     998112 non-null  float64
 9   ambient         998112 non-null  float64
 10  torque          998112 non-null  float64
 11  pm              998112 non-null  float64
dtypes: float64(12)
memory usage: 91.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332704 entries, 998112 to 1330815
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   --

In [ ]:
#Então aqui é colocado os valores de X_train como sendo todas as colunas menos a última, 
#isto é feito para isolar o valor Target que queremos predizer e que havia sido colocado para a última coluna
X_train = train_df.to_numpy()[:, :-1] 
#Aqui é capturada apenas a última coluna para termos o valor o qual o X_train deve alcançar.
y_train = train_df.to_numpy()[:, -1]
#Nesta parte, é feito exatamente a mesma coisa, porém agora sendo utilizado o conjunto de teste.
X_test = test_df.to_numpy()[:, :-1]
y_test = test_df.to_numpy()[:, -1] 

Agora que temos os dados tratados, será instalado o algorítmo TPOT que irá atuar na parte de treinamento do modelo.

In [ ]:
!pip install tpot
!pip install pydataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 5.6 MB/s 
     |████████████████████████████████| 160 kB 56.3 MB/s 
     |████████████████████████████████| 192.9 MB 74 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=f2c52676fe91278faa6a212fa460e5ec365b9d44c365e04b0726c6bec35b37d3
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.9 MB 18.9 MB/s 
  Created wheel for pydataset: filename=pydataset-0.2.0-py3-none-any.whl size=15939432 sha256=17db82e6fb901f079cbccf70bb9be0da72b24a539

Agora iremos trazer da biblioteca do TPOT a parte do modelo de regressão que ela oferece.



In [ ]:
from tpot import TPOTRegressor

Agora iremos utilizar TPOT Regressor para treinar nosso modelo

In [ ]:
#Neste momento estamos colocando os parâmetros ao TPOT
tpot = TPOTRegressor(generations=10,population_size=50,verbosity=2,random_state=5, max_time_mins=60)
#Agora iremos fazer o Fit de nossos dados ao modelo.
tpot.fit(X_train,y_train)
print(tpot.score(X_test,y_test))



Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]

In [ ]:
preds_test = tpot.predict(X_test)
print(r2_score(y_test, preds_test))

NameError: ignored